In [1]:
%run s0_fifa_players_functions.ipynb

In [2]:
goal_keepers = 'GK'
defenders = 'RWB', 'RCB', 'RB', 'LCB', 'CB', 'LWB', 'LB'
def_mid_fielders = 'LDM', 'CDM','RDM'
mid_fielders = 'CM', 'RCM', 'RM', 'LCM', 'LM'
attack_mid_fielders = 'LAM', 'CAM', 'RAM'
forwards = 'RF', 'CF','LF', 'LW', 'RW'
strikers = 'ST', 'RS', 'LS'

In [3]:
# Connect to DB
conn, cursor= connect()

import pandas
# sql = """ SELECT * from players where field_position in ('GK') """
sql = f"SELECT * from players where field_position in {attack_mid_fielders} "

attack_mid_fielders_data = pandas.read_sql(sql, con=conn)
# conn.commit()

# Disconnect from DB
disconnect(conn, cursor)

attack_mid_fielders_data = create_players_dataframes(attack_mid_fielders_data, columns_to_keep)
# display(goal_keepers_data)
attack_mid_fielders_data = create_fifa_players_dataset(attack_mid_fielders_data, "Resources/", "attack_mid_fielders.csv")

Connected!
Wall time: 39 ms


In [4]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import statsmodels.api as sm


In [5]:
file_path ='Resources/mid_fielders.csv'
attack_mid_fielders_df = pd.read_csv(file_path)
attack_mid_fielders_df['rating'] = (attack_mid_fielders_df['potential']+attack_mid_fielders_df['overall'])/2
attack_mid_fielders_df

,overall,potential,field_position,height,weight,crossing,finishing,heading_accuracy,short_passing,volleys,...,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,rating
0,91,92,RCM,5'11,154,93.0,82.0,55.0,92.0,82.0,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,91.5
1,91,91,RCM,5'8,146,86.0,72.0,55.0,93.0,76.0,...,84.0,60.0,76.0,73.0,13.0,9.0,7.0,14.0,9.0,91.0
2,90,90,LCM,6'0,168,88.0,76.0,54.0,92.0,82.0,...,85.0,72.0,79.0,69.0,10.0,11.0,13.0,7.0,10.0,90.0
3,90,90,LCM,5'8,148,84.0,76.0,54.0,93.0,82.0,...,93.0,59.0,53.0,29.0,6.0,15.0,7.0,6.0,12.0,90.0
4,88,95,RM,5'10,161,77.0,88.0,77.0,82.0,78.0,...,86.0,34.0,34.0,32.0,13.0,5.0,7.0,11.0,6.0,91.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,47,61,CM,5'7,146,31.0,28.0,40.0,53.0,31.0,...,35.0,41.0,44.0,54.0,10.0,12.0,6.0,11.0,8.0,54.0
4395,47,61,CM,5'10,141,44.0,44.0,36.0,53.0,43.0,...,52.0,41.0,47.0,38.0,13.0,6.0,9.0,10.0,15.0,54.0
4396,47,69,CM,5'6,150,31.0,31.0,41.0,51.0,26.0,...,40.0,48.0,49.0,49.0,7.0,7.0,9.0,14.0,15.0,58.0
4397,47,65,CM,5'9,134,34.0,38.0,40.0,49.0,25.0,...,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,56.0


In [6]:
list(attack_mid_fielders_df)

['overall',
 'potential',
 'field_position',
 'height',
 'weight',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'fk_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'composure',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'rating']

In [14]:
columns = ['field_position','height', 'weight', 'crossing', 'finishing', 'heading_accuracy',
           'short_passing', 'volleys', 'dribbling', 'curve', 'fk_accuracy', 'long_passing',
           'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
           'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
           'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'marking',
           'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking',
           'gk_positioning', 'gk_reflexes', 'rating']

attack_mid_fielders_df = attack_mid_fielders_df[columns]

attack_mid_fielders_df

,field_position,height,weight,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,...,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,rating
0,RCM,5'11,154,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,91.5
1,RCM,5'8,146,86.0,72.0,55.0,93.0,76.0,90.0,85.0,...,84.0,60.0,76.0,73.0,13.0,9.0,7.0,14.0,9.0,91.0
2,LCM,6'0,168,88.0,76.0,54.0,92.0,82.0,81.0,86.0,...,85.0,72.0,79.0,69.0,10.0,11.0,13.0,7.0,10.0,90.0
3,LCM,5'8,148,84.0,76.0,54.0,93.0,82.0,89.0,82.0,...,93.0,59.0,53.0,29.0,6.0,15.0,7.0,6.0,12.0,90.0
4,RM,5'10,161,77.0,88.0,77.0,82.0,78.0,90.0,77.0,...,86.0,34.0,34.0,32.0,13.0,5.0,7.0,11.0,6.0,91.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,CM,5'7,146,31.0,28.0,40.0,53.0,31.0,46.0,39.0,...,35.0,41.0,44.0,54.0,10.0,12.0,6.0,11.0,8.0,54.0
4395,CM,5'10,141,44.0,44.0,36.0,53.0,43.0,50.0,48.0,...,52.0,41.0,47.0,38.0,13.0,6.0,9.0,10.0,15.0,54.0
4396,CM,5'6,150,31.0,31.0,41.0,51.0,26.0,46.0,35.0,...,40.0,48.0,49.0,49.0,7.0,7.0,9.0,14.0,15.0,58.0
4397,CM,5'9,134,34.0,38.0,40.0,49.0,25.0,42.0,30.0,...,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,56.0


In [8]:
attack_mid_fielders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4399 entries, 0 to 4398
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   field_position    4399 non-null   object 
 1   height            4399 non-null   object 
 2   weight            4399 non-null   int64  
 3   crossing          4399 non-null   float64
 4   finishing         4399 non-null   float64
 5   heading_accuracy  4399 non-null   float64
 6   short_passing     4399 non-null   float64
 7   volleys           4399 non-null   float64
 8   dribbling         4399 non-null   float64
 9   curve             4399 non-null   float64
 10  fk_accuracy       4399 non-null   float64
 11  long_passing      4399 non-null   float64
 12  ball_control      4399 non-null   float64
 13  acceleration      4399 non-null   float64
 14  sprint_speed      4399 non-null   float64
 15  agility           4399 non-null   float64
 16  reactions         4399 non-null   float64


In [9]:
attack_mid_fielders_df.isna().any()

field_position      False
height              False
weight              False
crossing            False
finishing           False
heading_accuracy    False
short_passing       False
volleys             False
dribbling           False
curve               False
fk_accuracy         False
long_passing        False
ball_control        False
acceleration        False
sprint_speed        False
agility             False
reactions           False
balance             False
shot_power          False
jumping             False
stamina             False
strength            False
long_shots          False
aggression          False
interceptions       False
positioning         False
vision              False
penalties           False
composure           False
marking             False
standing_tackle     False
sliding_tackle      False
gk_diving           False
gk_handling         False
gk_kicking          False
gk_positioning      False
gk_reflexes         False
rating              False
dtype: bool

In [10]:
corr_matrix = attack_mid_fielders_df.corr()
corr_matrix['rating'].sort_values(ascending=False)


rating              1.000000
ball_control        0.877423
dribbling           0.825635
short_passing       0.769826
vision              0.757455
reactions           0.745510
composure           0.727828
positioning         0.697166
long_shots          0.664420
crossing            0.644424
shot_power          0.631903
long_passing        0.618726
curve               0.605818
finishing           0.602825
volleys             0.558764
fk_accuracy         0.486963
penalties           0.466502
stamina             0.428567
agility             0.376979
heading_accuracy    0.332194
aggression          0.295261
interceptions       0.293526
acceleration        0.289412
marking             0.278575
sprint_speed        0.251743
standing_tackle     0.240853
strength            0.207798
balance             0.193368
sliding_tackle      0.188668
jumping             0.142092
weight              0.100897
gk_kicking          0.047863
gk_reflexes         0.023338
gk_positioning      0.017064
gk_handling   

In [17]:
for position in attack_mid_fielders_df['field_position'].unique():
    
        try:
            # Subset by position
            temp_df = attack_mid_fielders_df[attack_mid_fielders_df["field_position"] == position]
            #temp_df = temp_df.dropna()
        
            # Find correlations
            temp_corr = attack_mid_fielders_df.corr()['rating'].to_frame()
            temp_corr = temp_corr[temp_corr['rating'] > 0.7]
            
            # Keep only columns with correltion above 0.7
            temp_features = list(temp_corr.T)
            temp_model_df = temp_df[temp_features]

            # Split data into train and test, isolate y (target/dependent variable)
            train_set, test_set = train_test_split(temp_model_df, test_size=0.2, random_state=42)
            target_train = train_set['rating']
            train_set = train_set.drop(['rating'], axis= 1) #'potential', 'overall'], axis = 1)
            
            # Fit model
            train_set = sm.add_constant(train_set, prepend=False)
            
            # Fit and summarize OLS model
            mod = sm.OLS(target_train, train_set)
            res = mod.fit()
            features = pd.DataFrame([res.pvalues, res.params]).T
            features.columns = ['P-value', 'Coefficient']
            features = features[features['P-value']<0.05]
            features = features.sort_values(by = 'Coefficient', ascending = False)
            
            
            print(f' ********* Current Position: {position}: \n ')
            print(f' ********* Significant features in order of impact: {features.index}: \n ')
            print(res.summary())
            print('\n')
       
        except BaseException as e:
            print(f'WARNING: SKIPPING {position} due to {e}')


 ********* Current Position: RCM: 
 
 ********* Significant features in order of impact: Index(['const', 'short_passing', 'reactions', 'ball_control', 'dribbling'], dtype='object'): 
 
                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     219.0
Date:                Wed, 14 Jul 2021   Prob (F-statistic):          2.15e-107
Time:                        19:31:15   Log-Likelihood:                -727.77
No. Observations:                 312   AIC:                             1470.
Df Residuals:                     305   BIC:                             1496.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef 